In [25]:
import os
from scipy.io import wavfile
import numpy as np
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as transforms
from IPython.display import Audio
# import ffmpeg
from IPython import display

print(torch.__version__)
print(torchaudio.__version__)

import matplotlib.pyplot as plt

2.1.0+cu121
2.1.0+cu121


### 멀티채널 데이터 만들기 (RIR + 소스)

In [26]:
# 지정 폴더 내 파일 개수 확인
folder1_path="Original"
folder2_path="RIR"

file1_list=os.listdir(folder1_path)

file2_src1_list=os.listdir(os.path.join(folder2_path, "src1"))
file2_src2_list=os.listdir(os.path.join(folder2_path, "src2"))

file1_count=len(file1_list)
file2_src1_count = len(file2_src1_list)
file2_src2_count = len(file2_src2_list)

print(file2_src1_count)
print(file2_src2_count)

6
6


In [27]:
# 모든 소스 파일 정렬
file1_list.sort()
print(file1_list)

['source1.wav', 'source2.wav']


In [28]:
# RIR 파일 정렬
file2_src1_list.sort()
file2_src2_list.sort()
print(file2_src1_list)
print(file2_src2_list)

['RIR_3ch_src1_rcv1.wav', 'RIR_3ch_src1_rcv2.wav', 'RIR_3ch_src1_rcv3.wav', 'RIR_3ch_src1_rcv4.wav', 'RIR_3ch_src1_rcv5.wav', 'RIR_3ch_src1_rcv6.wav']
['RIR_3ch_src2_rcv1.wav', 'RIR_3ch_src2_rcv2.wav', 'RIR_3ch_src2_rcv3.wav', 'RIR_3ch_src2_rcv4.wav', 'RIR_3ch_src2_rcv5.wav', 'RIR_3ch_src2_rcv6.wav']


In [29]:
folder3_path = "Convolution"
subfolder="src"
sample_rate = 48000

# RIR 6,6,3 / src1,2
src_idx=0
SAMPLE_SPEECH = os.path.join(folder1_path, file1_list[src_idx])
for rcv_idx in range(file2_src1_count):
        SAMPLE_RIR = os.path.join(folder2_path, subfolder+str(src_idx+1), file2_src1_list[rcv_idx])
        # Convolution 수행
        rir_raw, _ = torchaudio.load(SAMPLE_RIR)
        speech, _ = torchaudio.load(SAMPLE_SPEECH)
        augmented = F.fftconvolve(speech, rir_raw)
        filename = f"Convolution_3ch_src{src_idx+1}_rcv{rcv_idx+1}.wav"
        filepath = os.path.join(folder3_path, subfolder+str(src_idx+1), filename)
        # WAV 파일로 저장
        torchaudio.save(filepath, augmented, 48000)

src_idx=1
SAMPLE_SPEECH = os.path.join(folder1_path, file1_list[src_idx])
for rcv_idx in range(file2_src2_count):
        SAMPLE_RIR = os.path.join(folder2_path, subfolder+str(src_idx+1), file2_src2_list[rcv_idx])
        # Convolution 수행
        rir_raw, _ = torchaudio.load(SAMPLE_RIR)
        speech, _ = torchaudio.load(SAMPLE_SPEECH)
        augmented = F.fftconvolve(speech, rir_raw)
        filename = f"Convolution_3ch_src{src_idx+1}_rcv{rcv_idx+1}.wav"
        filepath = os.path.join(folder3_path, subfolder+str(src_idx+1), filename)
        # WAV 파일로 저장
        torchaudio.save(filepath, augmented, 48000)

print("Convolution wavfile 생성 완료")

Convolution wavfile 생성 완료


### 마이크 신호 만들기
- 마이크 3개 기준으로 소스 2개씩 합치기

In [30]:
# # 두 개의 WAV 파일 경로
# # mic1
# file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv1.wav")
# file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv1.wav")

# waveform1, _ = torchaudio.load(file1_path)
# waveform2, _ = torchaudio.load(file2_path)

# overlap_duration = min(waveform1.size(1), waveform2.size(1))
# overlap_waveform = waveform1[:, :overlap_duration] + waveform2[:, :overlap_duration]
# torchaudio.save(os.path.join(folder3_path,"mic_signal","room1","mic1.wav"), overlap_waveform, sample_rate)


# # mic2
# file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv2.wav")
# file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv2.wav")

# waveform1, _ = torchaudio.load(file1_path)
# waveform2, _ = torchaudio.load(file2_path)

# overlap_duration = min(waveform1.size(1), waveform2.size(1))
# overlap_waveform = waveform1[:, :overlap_duration] + waveform2[:, :overlap_duration]
# torchaudio.save(os.path.join(folder3_path,"mic_signal","room1","mic2.wav"), overlap_waveform, sample_rate)


# # mic3
# file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv3.wav")
# file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv3.wav")

# waveform1, _ = torchaudio.load(file1_path)
# waveform2, _ = torchaudio.load(file2_path)

# overlap_duration = min(waveform1.size(1), waveform2.size(1))
# overlap_waveform = waveform1[:, :overlap_duration] + waveform2[:, :overlap_duration]
# torchaudio.save(os.path.join(folder3_path,"mic_signal","room1","mic3.wav"), overlap_waveform, sample_rate)


- 더 긴 길이를 기준으로 overlap 해주고 짧은 길이는 그만큼 padding 해주고 싶음

In [31]:
file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv1.wav")
file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv1.wav")

waveform1, _ = torchaudio.load(file1_path)
waveform2, _ = torchaudio.load(file2_path)

waveform1 = torch.nn.functional.pad(waveform1,(0, int(1.0*sample_rate)))
waveform2 = torch.nn.functional.pad(waveform2,(int(1.0*sample_rate), 0))


min_length = min(waveform1.size(1), waveform2.size(1))
waveform1 = waveform1[:, :min_length]
waveform2 = waveform2[:, :min_length]
overlap_waveform = waveform1 + waveform2
torchaudio.save(os.path.join(folder3_path,"mic_signal","mic1.wav"), overlap_waveform, sample_rate)


# mic2
file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv2.wav")
file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv2.wav")

waveform1, _ = torchaudio.load(file1_path)
waveform2, _ = torchaudio.load(file2_path)

waveform1 = torch.nn.functional.pad(waveform1,(0, int(1.0*sample_rate)))
waveform2 = torch.nn.functional.pad(waveform2,(int(1.0*sample_rate), 0))

min_length = min(waveform1.size(1), waveform2.size(1))
waveform1 = waveform1[:, :min_length]
waveform2 = waveform2[:, :min_length]
overlap_waveform = waveform1 + waveform2
torchaudio.save(os.path.join(folder3_path,"mic_signal","mic2.wav"), overlap_waveform, sample_rate)


# mic3
file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv3.wav")
file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv3.wav")

waveform1, _ = torchaudio.load(file1_path)
waveform2, _ = torchaudio.load(file2_path)

waveform1 = torch.nn.functional.pad(waveform1,(0, int(1.0*sample_rate)))
waveform2 = torch.nn.functional.pad(waveform2,(int(1.0*sample_rate), 0))

min_length = min(waveform1.size(1), waveform2.size(1))
waveform1 = waveform1[:, :min_length]
waveform2 = waveform2[:, :min_length]
overlap_waveform = waveform1 + waveform2
torchaudio.save(os.path.join(folder3_path,"mic_signal","mic3.wav"), overlap_waveform, sample_rate)

In [32]:
file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv4.wav")
file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv4.wav")

waveform1, _ = torchaudio.load(file1_path)
waveform2, _ = torchaudio.load(file2_path)

waveform1 = torch.nn.functional.pad(waveform1,(0, int(1.0*sample_rate)))
waveform2 = torch.nn.functional.pad(waveform2,(int(1.0*sample_rate), 0))


min_length = min(waveform1.size(1), waveform2.size(1))
waveform1 = waveform1[:, :min_length]
waveform2 = waveform2[:, :min_length]
overlap_waveform = waveform1 + waveform2
torchaudio.save(os.path.join(folder3_path,"mic_signal","mic4.wav"), overlap_waveform, sample_rate)


# mic2
file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv5.wav")
file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv5.wav")

waveform1, _ = torchaudio.load(file1_path)
waveform2, _ = torchaudio.load(file2_path)

waveform1 = torch.nn.functional.pad(waveform1,(0, int(1.0*sample_rate)))
waveform2 = torch.nn.functional.pad(waveform2,(int(1.0*sample_rate), 0))

min_length = min(waveform1.size(1), waveform2.size(1))
waveform1 = waveform1[:, :min_length]
waveform2 = waveform2[:, :min_length]
overlap_waveform = waveform1 + waveform2
torchaudio.save(os.path.join(folder3_path,"mic_signal","mic5.wav"), overlap_waveform, sample_rate)


# mic3
file1_path = os.path.join(folder3_path,"src1","Convolution_3ch_src1_rcv6.wav")
file2_path = os.path.join(folder3_path,"src2","Convolution_3ch_src2_rcv6.wav")

waveform1, _ = torchaudio.load(file1_path)
waveform2, _ = torchaudio.load(file2_path)

waveform1 = torch.nn.functional.pad(waveform1,(0, int(1.0*sample_rate)))
waveform2 = torch.nn.functional.pad(waveform2,(int(1.0*sample_rate), 0))

min_length = min(waveform1.size(1), waveform2.size(1))
waveform1 = waveform1[:, :min_length]
waveform2 = waveform2[:, :min_length]
overlap_waveform = waveform1 + waveform2
torchaudio.save(os.path.join(folder3_path,"mic_signal","mic6.wav"), overlap_waveform, sample_rate)